In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import gensim.downloader as api
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

# Current work
print(os.getcwd())
# Change directory 2 level up
os.chdir("..")
os.chdir("..")
os.chdir("..")

print(os.getcwd())
# Look for the file in the directory
print(os.listdir())

/tmp/ipykernel_16096/3336268860.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


/home/jose/Git/SE-AP/experiments/experiment_2/notebooks
/home/jose/Git/SE-AP
['models', 'todo.md', '.pre-commit-config.yaml', 'demos', '.gitignore', 'experiments', 'include', '.git', '.env', 'README.md', '.env-example', 'LICENSE', 'model_technique_pathology_scores.csv']


In [2]:
model = "gemma"
prompt_technique = "react"
temperature = 0.9
data = pd.read_csv(f"experiments/experiment_2/results/{model}/results_{prompt_technique}_{model}_{temperature}_cleaned.csv")

In [3]:
grouped_data = data.groupby(['Induced Pathology', 'Iteration'])

unique_words_count = grouped_data['Surveyed Word'].nunique().reset_index(name='Unique Words Count')

total_rows = len(data)

expected_words_per_group = 160

valid_groups = unique_words_count['Unique Words Count'] == expected_words_per_group

all_groups_valid = valid_groups.all()

incorrect_groups = unique_words_count[~valid_groups]

incorrect_group_words = {}
for _, row in incorrect_groups.iterrows():
    pathology = row['Induced Pathology']
    iteration = row['Iteration']
    group_data = data[(data['Induced Pathology'] == pathology) & (data['Iteration'] == iteration)]
    duplicate_words = group_data[group_data.duplicated(subset=['Surveyed Word'], keep=False)]['Surveyed Word'].unique()
    incorrect_group_words[(pathology, iteration)] = list(duplicate_words)
    
result = {
    "total_rows": total_rows,
    "all_groups_have_160_unique_words": all_groups_valid,
    "groups_with_incorrect_word_counts": incorrect_groups.to_dict(orient='records'),
    "incorrect_group_words": incorrect_group_words
}

print(result)

{'total_rows': 159999, 'all_groups_have_160_unique_words': False, 'groups_with_incorrect_word_counts': [{'Induced Pathology': 'Alcohol Addiction', 'Iteration': 54.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Alcohol Addiction', 'Iteration': 59.0, 'Unique Words Count': 158}, {'Induced Pathology': 'Alcohol Addiction', 'Iteration': 86.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Apathy', 'Iteration': 6.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Apathy', 'Iteration': 38.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Apathy', 'Iteration': 63.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Apathy', 'Iteration': 87.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Depression', 'Iteration': 87.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Depression', 'Iteration': 88.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Impulsivity', 'Iteration': 79.0, 'Unique Words Count': 159}, {'Induced Pathology': 'Social Anxiety', 'Iteration': 88.0, 'U

In [4]:
grouped_by_category = data.groupby(['Induced Pathology', 'Iteration', 'Category'])
unique_words_per_category = grouped_by_category['Surveyed Word'].nunique().reset_index(name='Unique Words Count')

expected_words_per_category = 80
category_word_counts_valid = unique_words_per_category['Unique Words Count'] == expected_words_per_category

groups_with_incorrect_word_counts = unique_words_per_category[~category_word_counts_valid] if not category_word_counts_valid.all() else None

{
    "all_categories_have_80_unique_words": category_word_counts_valid.all(),
    "groups_with_incorrect_word_counts": groups_with_incorrect_word_counts
}


{'all_categories_have_80_unique_words': False,
 'groups_with_incorrect_word_counts':       Induced Pathology  Iteration  Category  Unique Words Count
 107   Alcohol Addiction       54.0  Concrete                  79
 116   Alcohol Addiction       59.0  Abstract                  79
 117   Alcohol Addiction       59.0  Concrete                  79
 170   Alcohol Addiction       86.0  Abstract                  79
 210              Apathy        6.0  Abstract                  79
 274              Apathy       38.0  Abstract                  79
 325              Apathy       63.0  Concrete                  79
 326              Apathy       63.0   Unknown                   0
 373              Apathy       87.0  Abstract                  79
 573          Depression       87.0  Abstract                  79
 575          Depression       88.0  Abstract                  79
 958         Impulsivity       79.0  Concrete                  79
 959         Impulsivity       79.0   Unknown             

In [5]:
concreteness = pd.read_json('experiments/experiment_2/data/concreteness.json')

missing_words_per_iteration = {}

for (pathology, iteration, category), df_group in data.groupby(['Induced Pathology', 'Iteration', 'Category']):
    key = f"{pathology} | Iteration: {iteration} | {category}"
    expected_words = concreteness.get(category, [])
    present_words = df_group['Surveyed Word'].unique()
    missing_words = [word for word in expected_words if word not in present_words]
    
    if missing_words:
        missing_words_per_iteration[key] = missing_words

list(missing_words_per_iteration.items())[:5]

[]